<a href="https://colab.research.google.com/github/unreal-kz/Notes/blob/master/FineTuneSAM_SEG_ECHO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install -q transformers datasets segments-ai evaluate
!pip install -U accelerate
!pip install -U transformers